In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.453254683870284, 7.224788339841421, 7.00834654020913]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="100um",
                                              pad_height="480um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [17]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [18]:
hfss.start()

INFO 11:54PM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:54PM [load_ansys_project]: 	Opened Ansys App
INFO 11:54PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:54PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project23
INFO 11:54PM [connect_design]: No active design found (or error getting active design).
INFO 11:54PM [connect]: 	 Connected to project "Project23". No design detected


True

In [19]:
hfss.activate_ansys_design("flip_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

11:54PM 25s WARNING [activate_ansys_design]: The design_name=flip_V6 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:54PM [connect_design]: 	Opened active design
	Design:    flip_V6 [Solution type: Eigenmode]
WARNING 11:54PM [connect_setup]: 	No design setup detected.
WARNING 11:54PM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:54PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [12]:
hfss.close()

NameError: name 'hfss' is not defined

# LOM Analysis

In [13]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [14]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.58',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.58',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [17]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 12:37AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp1jqxgr1x.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5580000000, Maxwell, 1, False
INFO 12:37AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpvhm91vuf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5580000000, Maxwell, 1, False
INFO 12:37AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpos9leqpg.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5580000000, Maxwell, 2, False
INFO 12:37AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpln07o2q4.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5580000000, Maxwell, 3, False
INFO 12:37AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpmpngi8hn.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5580000000, Maxwell, 4, False
INFO 12:37

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,376.660,-74.67600,-55.969,-186.11000
pad_bot_Q0,-74.676,107.85000,-31.533,-0.54124
pad_top_Q0,-55.969,-31.53300,107.500,-18.86300
quad_coupler_c0,-186.110,-0.54124,-18.863,206.44000


In [18]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.58
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.405509 [GHz]
EC 272.648433 [MHz]
EJ 14.854145 [GHz]
alpha -310.879041 [MHz]
dispersion 5.006355 [KHz]
Lq 10.995564 [nH]
Cq 71.044707 [fF]
T1 26.745653 [us]

**Coupling Properties**

tCqbus1 -9.186045 [fF]
gbus1_in_MHz -123.825497 [MHz]
χ_bus1 -5.532717 [MHz]
1/T1bus1 5950.684546 [Hz]
T1bus1 26.745653 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.541839,287.444695,14.854145,-329.201848,8.475351,[-10.809800010648587],[-0.05473566535976251],0.054736,10.809800
2,5.615903,295.669074,14.854145,-339.447235,11.159433,[-188.39885096256256],[-19.340637986021463],19.340638,188.398851
3,5.560149,289.465739,14.854145,-331.715497,9.078172,[-177.20004171246325],[-15.258275603924261],15.258276,177.200042
4,5.570122,290.569808,14.854145,-333.089779,9.422502,[-157.78102903697155],[-12.343810203837672],12.343810,157.781029
5,5.530046,286.147283,14.854145,-327.589591,8.106482,[-140.1257763576899],[-8.984636079595873],8.984636,140.125776
6,5.479673,280.642908,14.854145,-320.761482,6.6886,[-130.2843290976489],[-7.040842946590438],7.040843,130.284329
7,5.452026,277.647398,14.854145,-317.053758,6.00952,[-127.33191040312377],[-6.38050690545699],6.380507,127.331910
8,5.434289,275.735243,14.854145,-314.689975,5.60736,[-124.48387039246356],[-5.898355498491929],5.898355,124.483870
9,5.416216,273.794543,14.854145,-312.293301,5.222778,[-124.36825134699436],[-5.692731360153466],5.692731,124.368251
10,5.405509,272.648433,14.854145,-310.879041,5.006355,[-123.82549726407754],[-5.532716776504408],5.532717,123.825497


In [19]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
